In [1]:
import numpy as np
import open3d as o3d
import os 
import pdal 
import json 
import time
import copy 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
templatefolder = "templates"
datafolder = "template_versuchsdaten"

In [3]:
os.listdir(templatefolder) 

['T_profil_ähnlich_54E4_breiter_schaft_schräg_random.ply',
 'T_profil_ähnlich_54E4_breiter_schaft.ply',
 'T_profil_ähnlich_54E4_breiter_schaft_wenig_punkte_schräg_random.ply',
 'T_profil_ähnlich_54E4_breiter_schaft_schräg.ply',
 'T_profil_ähnlich_54E4_schräg_random.ply',
 'T_profil_ähnlich_54E4.ply',
 'T_profil_ähnlich_54E4_schräg.ply']

In [23]:
template_file = os.listdir(templatefolder)[2]
print(template_file)
template_file = os.path.join(templatefolder, template_file) 

template = o3d.io.read_point_cloud(template_file) 

T_profil_ähnlich_54E4_breiter_schaft_wenig_punkte_schräg_random.ply


In [5]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

In [63]:
os.listdir(datafolder)

['abseits4.las',
 'anfang_weiche_kästen.laz',
 'abseits3.las',
 'nebengleis2.las',
 'abseits5.las',
 'einfach3.las',
 'einfach4.las',
 'abseits1.las',
 'einfach_kasten.las',
 'einfach2.las',
 'einfach_und_schrott.las',
 'nicht_ganz_einfach.las',
 'weicheB_X.laz',
 'weicheB_TTTTT.laz',
 'weicheB_IIIII.laz',
 'kein_gleis.las',
 'abseits2.las',
 'abseits6.las',
 'befahren_links.las',
 'ganz',
 'nur_schotter.laz',
 'einfach_nix.las',
 'befahren_rechts.las',
 'einfach1.las',
 'anfang_weiche_2eng.laz',
 'weicheB_Y.laz',
 'anfang_weiche_2extremeng.laz',
 'nebengleis1.las']

In [6]:
cloud_file = 'nebengleis1.las'


cloud_file = os.path.join(datafolder, cloud_file)
if not os.path.exists(cloud_file):
    raise FileNotFoundError(cloud_file) 

pipeline = pdal.Pipeline([pdal.Reader(cloud_file)])
pipeline.execute()
points = pipeline.arrays[0]

In [7]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
rgb = np.vstack((points['Red'], points['Green'], points['Blue'])).transpose() / 65535.0

offset = xyz.mean(axis=0).round() 
xyz -= offset

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)

In [8]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.6, origin=[0, 0, 0])

In [9]:
o3d.visualization.draw_geometries([pcd, template, mesh_frame], front=front, lookat=lookat, up=up, zoom=zoom)

## Fast Global Registration
source = bewegt, target = fest

In [24]:
# Options 
normals_radius = 0.1
normals_max_nn = 30

fpfh_radius = 0.2
fpfh_max_nn = 100

distance_threshold = 0.025

In [25]:
template.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=normals_radius, max_nn=normals_max_nn))
template_fpfh = o3d.pipelines.registration.compute_fpfh_feature(template, o3d.geometry.KDTreeSearchParamHybrid(radius=fpfh_radius, max_nn=fpfh_max_nn))

In [26]:
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=normals_radius, max_nn=normals_max_nn))
pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=fpfh_radius, max_nn=fpfh_max_nn))

Pcd als source, template als Target

In [27]:
start = time.time()

# source, target, source_features, target_features, option
result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
    pcd, template, pcd_fpfh, template_fpfh,
    o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))

print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result)
print(result.transformation)

source_temp = copy.deepcopy(pcd)
source_temp.transform(result.transformation)
o3d.visualization.draw_geometries([source_temp, template, mesh_frame], front=front, lookat=lookat, up=up, zoom=zoom)

Fast global registration took 0.173 sec.

RegistrationResult with fitness=2.489960e-02, inlier_rmse=1.469646e-02, and correspondence_set size of 62
Access transformation to get result.
[[ 1.         -0.          0.         -0.11228624]
 [-0.          1.         -0.         -0.49762755]
 [ 0.         -0.          1.          0.42031909]
 [-0.          0.         -0.          1.        ]]


Template als Source, PCD als target

In [28]:
start = time.time()

# source, target, source_features, target_features, option
result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
    template, pcd, template_fpfh, pcd_fpfh,
    o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))

print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result)
print(result.transformation)

source_temp = copy.deepcopy(template)
source_temp.transform(result.transformation)
o3d.visualization.draw_geometries([source_temp, pcd, mesh_frame], front=front, lookat=lookat, up=up, zoom=zoom)

Fast global registration took 0.124 sec.

RegistrationResult with fitness=2.011334e-01, inlier_rmse=1.767146e-02, and correspondence_set size of 2307
Access transformation to get result.
[[ 1.         -0.          0.          0.11228624]
 [-0.          1.         -0.          0.49762755]
 [-0.          0.          1.         -0.42031909]
 [-0.          0.         -0.          1.        ]]


## PCA

In [15]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    # Vectors are columns
    return sorted_eigenvals, sorted_eigenvecs

def linearity(eigenvals):
    """Calculate the linearity of a point cloud"""
    return (eigenvals[0] - eigenvals[1]) / eigenvals[0]


def theta(eigenvects):
    """Angle between the first eigenvector and the z-axis"""
    cos_theta = eigenvects.T[0] @ np.array([0, 0, 1]) / np.linalg.norm(eigenvects[0])
    return np.arccos(cos_theta) * 180 / np.pi

In [16]:
def plot_vector(vector, stretch=1):
    """Plot a vector as a line"""
    vector = vector * stretch
    line = o3d.geometry.LineSet()
    line.points = o3d.utility.Vector3dVector([[0, 0, 0], vector])
    line.lines = o3d.utility.Vector2iVector([[0, 1]])
    line.colors = o3d.utility.Vector3dVector([[1, 0, 0]])
    return line

In [17]:
def plot_eigenvectors(eigenvects):
    """Plot the eigenvectors of a point cloud"""
    eigenvects = eigenvects.T
    lines = o3d.geometry.LineSet()
    lines.points = o3d.utility.Vector3dVector([[0, 0, 0], eigenvects[0], eigenvects[1], eigenvects[2]])
    lines.lines = o3d.utility.Vector2iVector([[0, 1], [0, 2], [0, 3]])
    lines.colors = o3d.utility.Vector3dVector([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    return lines

In [18]:
eigenvals, eigenvecs = pca(xyz)

print("Linearity: ", linearity(eigenvals))
print("Theta: ", theta(eigenvecs))

Linearity:  0.9998924848431279
Theta:  90.5483385095337


In [19]:
template_eigenvals, template_eigenvecs = pca(np.asarray(template.points))
print("Template Linearity: ", linearity(template_eigenvals))
print("Template Theta: ", theta(template_eigenvecs))

Template Linearity:  0.875503320752718
Template Theta:  90.0


In [20]:
lines = plot_eigenvectors(template_eigenvecs)

o3d.visualization.draw_geometries([pcd, template, mesh_frame, lines], front=front, lookat=lookat, up=up, zoom=zoom)

In [21]:
eigenvals 

array([4.02896618e+00, 4.33174931e-04, 2.19121883e-04])

In [22]:
template_eigenvals

array([0.02083399, 0.00259376, 0.00051827])